In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA

# Azure data
### Write more

In [2]:
azure_data=pd.read_pickle('Azure_Alibaba.pkl')
endvalue=azure_data.shape[0]
azure_data.shape

(8571, 1)

# Alibaba Data
### Write more

In [3]:
alibaba_data=pd.read_pickle('Alibaba_Azure.pkl')
alibaba_data=alibaba_data[0:8751]
alibaba_data.shape

(8751, 1)

In [4]:
azure_data=np.array(azure_data,dtype=float)
azure_data=np.ravel(azure_data)
M=np.amin(azure_data)
azure_data.shape

(8571,)

In [5]:
alibaba_data=np.array(alibaba_data,dtype=float)
alibaba_data=np.ravel(alibaba_data)
m=np.amin(alibaba_data)
alibaba_data.shape

(8751,)

## Removing the NaN values

In [6]:
azure_data=np.nan_to_num(azure_data,nan=M)

In [7]:
alibaba_data=np.nan_to_num(alibaba_data,nan=m)

## Combining the Array

In [8]:
combined=np.append(azure_data,alibaba_data)

## Generating Training and Test data
### Train and Test data is generated according to the shape of the traces

In [9]:
train= combined[0:endvalue]


## Function to get valid list

In [10]:
def try_forecast_model(p,q,d):
    temp=[]
    model=ARIMA(train,order=(p,q,d))
    try:
        model_fit=model.fit(disp=0)
    except:
        temp.append(p)
        temp.append(q)
        temp.append(d)
        invalid_list.append(temp)

## Declaring 'p','q','d' values for the model
### These values are being declared to do a sort of grid search

In [11]:
AR_parameter=[9]
MA_parameter=[1]
difference=[3]

In [12]:
total_list=[]
invalid_list=[]
for p in AR_parameter:
    for q in MA_parameter:
        for d in difference:
            initial_temp=[]
            initial_temp.append(p)
            initial_temp.append(q)
            initial_temp.append(d)
            total_list.append(initial_temp)
            try_forecast_model(p,q,d)

/home/abhilash/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


## Finding a valid list of hyperparameters

In [13]:
valid_list = []
for list in total_list:
    if list not in invalid_list:
        valid_list.append(list) 

## Forecasting Model

In [14]:
rmse_accuracy=[]
predicted=[]
mape=[]
def forecast_model(p,q,d):
    temp=[]
    model=ARIMA(train,order=(p,q,d))
    model_fit=model.fit(disp=0)
    end_values=[10,15,50,100]
    for value in end_values:
        forecast=model_fit.predict(start=endvalue,end=endvalue+value)
        predicted.append(forecast)
        actual_values=combined[endvalue:(endvalue+value+1)]
        accuracy=np.mean((forecast - actual_values)**2)**.5
        rmse_accuracy.append(accuracy)
        acc = np.mean(np.abs(forecast - actual_values)/np.abs(actual_values))
        mape.append(acc)

### Running the model

In [15]:
for val in valid_list:
    forecast_model(val[0],val[1],val[2])

/home/abhilash/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [16]:
rmse_accuracy

[0.05785740684886375, 0.058245153342476104, nan, nan]

In [17]:
mape

[0.999469084973104, 0.9996391917529588, nan, nan]

In [18]:
maximum=np.amax(rmse_accuracy)

In [19]:
ind=0
for i in range(0,len(rmse_accuracy)):
    if rmse_accuracy[i] == maximum:
        ind=i
        break

In [20]:
ind

179

In [21]:
mape[ind]

inf

In [22]:
rmse_accuracy[ind]

0.06770203318586608

In [23]:
valid_list[45]

[9, 1, 3]

In [24]:
np.save('AzureAlibaba_predicted.npy',predicted[ind])